In [1]:
import pandas as pd
import regex as re
import spacy
spacy.require_gpu()
from torch.utils import dlpack

In [2]:
import importlib
import sys

spec_src = importlib.util.spec_from_file_location(
    'src', 
    '../../__init__.py')
m = importlib.util.module_from_spec(spec_src)
sys.modules[spec_src.name] = m
spec_src.loader.exec_module(m)

from src import utils

In [3]:
conf = utils.get_config('p')

In [4]:
data = utils.get_csv('twitter', utils.get_save_path('p', 'twitter', lang='es')/'2022-02-08 16:34:36'/'20210726'/'acordar'/'sample-acordar-tweets-processed.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20945 entries, 0 to 20944
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   verbs         20945 non-null  string 
 1   tweet_id      20945 non-null  float64
 2   text_orig     20945 non-null  string 
 3   normalized    20945 non-null  object 
 4   has_ccomp     20945 non-null  bool   
 5   dependencies  20945 non-null  string 
 6   pos           20945 non-null  object 
 7   details       20945 non-null  object 
dtypes: bool(1), float64(1), object(3), string(3)
memory usage: 1.1+ MB


In [5]:
data['pos'].head()

0    @MicaaFerreiraa(PROPN) Si(INTJ) si(SCONJ) se(P...
1    Nunca(ADV) he(AUX) escuchado(VERB) ninguna(DET...
2    ESPNF90(PROPN) van(AUX) a(ADP) hablar(VERB) to...
3    Lo(PRON) único(ADJ) que(PRON) quiero(VERB) es(...
4    @PauMatalap(PROPN) hospital(NOUN) Pues(SCONJ) ...
Name: pos, dtype: object

In [6]:
pos_pat = r'\(\bAUX\b\)|\(\bVERB\b\)'

In [7]:
# TODO: improve by checking the word's lemma to ensure any 'AUX' is a verb

def get_consecutive_verbs(text: str) -> (int, list):
    """
    Count instances of consecutive verbs 
    
    :param text: input text string
    
    :return: tuple (num of consecutive verb groups, list of consecutive groups)
    """
    
    text = text.split()
    
    # Holder for consecutive group
    consec = []
    # All consecutive verbs
    cverbs = []
    
    for word in text:
        match = re.search(pos_pat, word)
        
        if match is None:
            consec = []
            continue
            
        mat_group = match.group(0)
        
        if len(mat_group) != 0:
            consec.append(word)
            
            # Non-verb; if holder has >= 2 verbs, record it
            if len(consec) > 1:
                cverbs.append(consec)
            continue
    
    return len(cverbs), cverbs

In [8]:
def test_func(func, idx, correct, **params):
    out = func(**params)
    
    print(f'Function {func.__name__}(), returned: \n\t{out}')
    print(f'Desired output: \n\t{correct}')
    
    if idx != -1:
        out = func(**params)[idx]
    
    return out == correct

In [14]:
test_input = data.sample(5, random_state=1).loc[:, ['tweet_id', 'text_orig', 'pos', 'details']]
test_input.head()

,tweet_id,text_orig,pos,details
10734,1.419572e+18,Casi todo salió bien hoy y no sé qué más le po...,Casi(ADV) todo(PRON) salió(VERB) bien(ADV) hoy...,"<Casi>(casi,True) <todo>(todo,True) <salió>(sa..."
1084,1.419547e+18,A veces vengo disque a twittear algo para ti p...,A(ADP) veces(NOUN) vengo(VERB) disque(NOUN) a(...,"<A>(a,False) <veces>(veces,True) <vengo>(venir..."
3484,1.419739e+18,@rosadiezglez ¿Por qué hablas en nombre de los...,@rosadiezglez(PROPN) Por(ADP) qué(PRON) hablas...,"<@rosadiezglez>(@rosadiezglez,False) <Por>(por..."
13829,1.419744e+18,@todoliderazgo Muy preocupante todo esto. Y mu...,@todoliderazgo(PROPN) Muy(ADV) preocupante(ADJ...,"<@todoliderazgo>(@todoliderazgo,False) <Muy>(m..."
4707,1.419445e+18,@elespectador @IvanDuque El espíritu de escuch...,@elespectador(NOUN) @IvanDuque(PROPN) El(DET) ...,"<@elespectador>(@elespectador,False) <@IvanDuq..."


In [18]:
with pd.option_context('display.max_colwidth', None):
    display(test_input['text_orig'])
    display(test_input['pos'])

10734                                                                                        Casi todo salió bien hoy y no sé qué más le podría pedir a la vida. Tengo gente súper rifada en cocina y amigues que están dispuestxs a partírsela por mí. Blessed af.
1084                                                                                                        A veces vengo disque a twittear algo para ti pero si que sepas que es para ti y luego que lo escribo, noto que soy muy evidente y lo termino borrando 😶
3484                                                                                                                   @rosadiezglez ¿Por qué hablas en nombre de los españoles? Los españoles dejaron de votar te hace mucho ya, no creo que representes a muchos.
13829    @todoliderazgo Muy preocupante todo esto. Y muy preocupante que la mayoría lo vea como normal y justo. En Francia 74 diputados han recurrido la medida al Constitucional, en Italia la gente empieza a manifestar s

10734                                                                                                                                 Casi(ADV) todo(PRON) salió(VERB) bien(ADV) hoy(ADV) y(CCONJ) no(ADV) sé(VERB) qué(PRON) más(ADV) le(PRON) podría(AUX) pedir(VERB) a(ADP) la(DET) vida(NOUN) Tengo(VERB) gente(NOUN) súper(ADJ) rifada(ADJ) en(ADP) cocina(NOUN) y(CCONJ) amigues(NOUN) que(PRON) están(AUX) dispuestxs(ADJ) a(ADP) partírsela(VERB) por(ADP) mí(PRON) Blessed(PROPN) af(ADV)
1084                                                                                                                                                 A(ADP) veces(NOUN) vengo(VERB) disque(NOUN) a(ADP) twittear(VERB) algo(PRON) para(ADP) ti(PRON) pero(CCONJ) si(SCONJ) que(SCONJ) sepas(VERB) que(SCONJ) es(AUX) para(ADP) ti(PRON) y(CCONJ) luego(ADV) que(SCONJ) lo(PRON) escribo(VERB) noto(VERB) que(SCONJ) soy(AUX) muy(ADV) evidente(ADJ) y(CCONJ) lo(PRON) termino(VERB) borrando(VERB)
3484                              

In [16]:
correct_out = [1, 0, 0, 1, 1]

In [17]:
for i, t in enumerate(test_input.iterrows()):
    entry = t[1]
    
    print(f'!!!Entry {entry["tweet_id"]}!!!')
    print(test_func(get_consecutive_verbs, 0, correct=correct_out[i], text=entry['pos']))
    print('\n')

!!!Entry 1.4195715185902633e+18!!!
Function get_consecutive_verbs(), returned: 
	(1, [['podría(AUX)', 'pedir(VERB)']])
Desired output: 
	1
True


!!!Entry 1.419547490580316e+18!!!
Function get_consecutive_verbs(), returned: 
	(2, [['escribo(VERB)', 'noto(VERB)'], ['termino(VERB)', 'borrando(VERB)']])
Desired output: 
	0
False


!!!Entry 1.419738680009298e+18!!!
Function get_consecutive_verbs(), returned: 
	(0, [])
Desired output: 
	0
True


!!!Entry 1.419743930279682e+18!!!
Function get_consecutive_verbs(), returned: 
	(1, [['han(AUX)', 'recurrido(VERB)']])
Desired output: 
	1
True


!!!Entry 1.419444543095722e+18!!!
Function get_consecutive_verbs(), returned: 
	(1, [['dialogar(VERB)', 'está(VERB)']])
Desired output: 
	1
True


